In [ ]:
!pip install -q datasets
!pip install -q -U bitsandbytes
!pip install -q -U flash_attn
!pip install -q evaluate
!pip install -q rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import pipeline, BitsAndBytesConfig
import bitsandbytes as bnb
import random
import evaluate
import pandas as pd
from tqdm import tqdm
from pprint import pprint

In [ ]:
# Set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)

ds = load_dataset("rony/climate-change-MRC")

README.md:   0%|          | 0.00/633 [00:00<?, ?B/s]

CCMRC_train.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

CCMRC_validation.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

CCMRC_test.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
def prepare_data(data):
  articles = []

  for paragraph in data:
    context = paragraph['context']
    for qa in paragraph['qas']:
      question = qa['question']
      id = qa['id']
      for ans in qa['answers']:
        answer = ans['text']
        answer_start = ans['answer_start']
        articles.append({'context': context, 'question': question, 'id': id, 'answer': answer, 'answer_start': answer_start})

  return articles

In [ ]:
train_ds = ds["train"]
valid_ds = ds["validation"]
test_ds = ds["test"]

# each is a 1-item list, so take first index
train_ds = train_ds[0]
valid_ds = valid_ds[0]
test_ds = test_ds[0]

# take the 'data' key of the dict, ignoring 'version' (there's just one)
train_ds = train_ds['data'][0]['paragraphs']
valid_ds = valid_ds['data'][0]['paragraphs']
test_ds = test_ds['data'][0]['paragraphs']
# each dataset is a list of dicts, where each list item is a context paragraph ('context' key) with qas ('qas' key) which contain questions, id, and answer

train_df = pd.DataFrame(prepare_data(train_ds))
print(f"{train_df.shape=}")

valid_df = pd.DataFrame(prepare_data(valid_ds))
print(f"{valid_df.shape=}")

test_df = pd.DataFrame(prepare_data(test_ds))
print(f"{test_df.shape=}")

train_df.shape=(14756, 5)
valid_df.shape=(4229, 5)
test_df.shape=(2096, 5)


In [ ]:
train_df.head()

,context,question,id,answer,answer_start
0,"outside of the clean air act, there is support...",Is a mandate for electric production to come f...,665,"and there is ongoing interest in a ""national r...",222
1,"outside of the clean air act, there is support...",How wide ranging are climate policies across t...,666,"as of 2010, climate policies were being contem...",618
2,"outside of the clean air act, there is support...",What kind of energy standards have been implem...,667,"at latest count, 30 states have implemented re...",734
3,acknowledgments. we thank jim haywood for the ...,What was Jim Haywood thanked for?,1731,we thank jim haywood for the aerosol forcing pdf,17
4,acknowledgments. we thank jim haywood for the ...,Where was Jonathan Gregory supported?,1732,jonathan gregory was supported at the universi...,277


In [ ]:
"""
Initialize the pipeline with bitsandbytes quantization
"""
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Initialize pipeline
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

summarizer = pipeline(
   "text-generation",
   model=model_id,
   model_kwargs={"torch_dtype": torch.bfloat16, "quantization_config": quantization_config},
   device_map="auto",
   trust_remote_code=True,
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
prompt = [
            {"role": "system", "content": "You are skilled in answering questions."},
            {"role": "user", "content": "Generate an answer to this question using the provided context. The format will be the context followed by the question: "},
            {"role": "user", "content": test_df['context'][0]},
            {"role": "user", "content": test_df['question'][0]}
        ]


outputs = summarizer(
  prompt,
  max_new_tokens=512,
  do_sample = True,
  temperature = 0.3,
  top_p = 0.95
)

answer = outputs[0]["generated_text"][-1]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
answer

{'role': 'assistant',
 'content': 'The 9 percent reduction of rice in Bangladesh is attributed to two variables: \n\n1. Flooding damage\n2. Climate variability'}

In [ ]:
test_df['answer'][0]

'flooding damage and climate variability'

In [ ]:
rouge = evaluate.load('rouge')

# Calculate ROUGE scores
predictions = [answer['content']]
references = [test_df['answer'][0]]
rouge_scores = rouge.compute(predictions=predictions, references=references)
rouge_scores

{'rouge1': np.float64(0.3333333333333333),
 'rouge2': np.float64(0.1818181818181818),
 'rougeL': np.float64(0.3333333333333333),
 'rougeLsum': np.float64(0.3333333333333333)}

In [ ]:
results = []

In [ ]:
for idx, sample in tqdm(test_df.iterrows()):
    try:
      prompt = [
            {"role": "system", "content": "You are skilled in answering questions."},
            {"role": "user", "content": "Generate an answer to this question using the provided context. The format will be the context followed by the question: "},
            {"role": "user", "content": test_df['context'][idx]},
            {"role": "user", "content": test_df['question'][idx]}
      ]

      # Generate summary via the pipeline
      outputs = summarizer(
                          prompt,
                          max_new_tokens=512,
                          do_sample = True,
                          pad_token_id = summarizer.tokenizer.eos_token_id,
                          temperature = 0.3,
                          top_p = 0.95
      )

      answer = outputs[0]["generated_text"][-1]

      # Calculate ROUGE scores
      predictions = [answer['content']]
      references = [[sample['answer']]]
      rouge_scores = rouge.compute(predictions=predictions, references=references)


      # Store results
      results.append({
          'id': idx,
          'original_context': sample['context'][:500],
          'original_question': sample['question'],
          'reference_answer': sample['answer'],
          'generated_answer': answer,
           **rouge_scores
      })

      # Print progress update every 200 samples
      if (idx + 1) % 200 == 0:
          print(f"\nProcessed {idx + 1} samples")
          print(f"Latest ROUGE-1: {rouge_scores['rouge1']:.4f}")

    except Exception as e:
      print(f"Error processing sample {idx}: {str(e)}")
      continue

200it [31:31, 10.29s/it]


Processed 200 samples
Latest ROUGE-1: 0.5357


400it [1:06:16, 10.84s/it]


Processed 400 samples
Latest ROUGE-1: 0.9455


502it [1:20:51,  9.66s/it]


KeyboardInterrupt: 

In [ ]:
# results for first 10 iterations
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Calculate and print average ROUGE scores
avg_scores = results_df[['rouge1', 'rouge2', 'rougeL']].mean()
print("\nAverage ROUGE Scores:")
for metric, score in avg_scores.items():
   print(f"{metric}: {score:.4f}")

# Print some example answers
print("\nExample Summaries:")
for i in range(min(5, len(results_df))):
   print(f"\nExample {i+1}:")
   print(f"Reference: {results_df.iloc[i]['reference_answer']}")
   print(f"Generated: {results_df.iloc[i]['generated_answer']}")


Average ROUGE Scores:
rouge1: 0.5633
rouge2: 0.4852
rougeL: 0.5332

Example Summaries:

Example 1:
Reference: flooding damage and climate variability
Generated: {'role': 'assistant', 'content': 'According to the text, the 9 percent reduction of rice production in Bangladesh is attributed to two variables:\n\n1. Flooding damage\n2. Climate variability'}

Example 2:
Reference: a dynamic economywide model
Generated: {'role': 'assistant', 'content': 'A dynamic economy-wide model of Bangladesh was used to estimate economic damages from historical climate variability and future anthropogenic climate change.'}

Example 3:
Reference: a modelling approach
Generated: {'role': 'assistant', 'content': 'According to the text, Ahmed et al. (2011) used a modelling approach to estimate how changes in climate variability might affect crop yields and thence poverty rates in Tanzania to the early 2030s.'}

Example 4:
Reference: extreme sea level height fluctuations are also larger to the north, as a res

In [ ]:
# results for 502 iterations
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Calculate and print average ROUGE scores
avg_scores = results_df[['rouge1', 'rouge2', 'rougeL']].mean()
print("\nAverage ROUGE Scores:")
for metric, score in avg_scores.items():
   print(f"{metric}: {score:.4f}")

# Print some example answers
print("\nExample Summaries:")
for i in range(200, 210):
   print(f"\nExample {i+1}:")
   print(f"Reference: {results_df.iloc[i]['reference_answer']}")
   print(f"Generated: {results_df.iloc[i]['generated_answer']}")


Average ROUGE Scores:
rouge1: 0.5096
rouge2: 0.4167
rougeL: 0.4696

Example Summaries:

Example 201:
Reference: models and empirical studies suggest that the speed with which a population adapts to a changing environment affects invasion rate of new habitat and thus migration rate, population growth rate and thus probability of extinction, and growth and mortality of individual plants and thus productivity of regional vegetation
Generated: {'role': 'assistant', 'content': 'According to the text, the models and empirical studies revealed that the speed with which a population adapts to a changing environment affects several key factors, including:\n\n1. Invasion rate of new habitat\n2. Migration rate\n3. Population growth rate and thus probability of extinction\n4. Growth and mortality of individual plants and thus productivity of regional vegetation.'}

Example 202:
Reference: as this study covers a large spatial scale, we anticipate that results are also valid for other regions
Gener